In [1]:
import pandas as pd
import random
from time import sleep
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import numpy as np
import nltk
import os
from sklearn import datasets
import pickle
from nltk.tokenize import word_tokenize
from sklearn.decomposition import NMF, TruncatedSVD
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.neighbors import NearestNeighbors
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, roc_curve, auc, fbeta_score
from sklearn.model_selection import learning_curve, KFold, GridSearchCV, cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.dummy import DummyClassifier
from collections import Counter
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.preprocessing import normalize
import re
from unidecode import unidecode

import matplotlib.style as style
%matplotlib inline
plt.style.use('ggplot')

/Users/brendenrossin/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
with open('data/cleaned_df.pkl', 'rb') as picklefile:
    df = pickle.load(picklefile)

In [3]:
df.head()

,song_owner,song_title,song_element,verse,artist
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],"We just wanna party, Party just for you, We ju...",Childish Gambino
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],"This is America, Don't catch you slippin' now,...",Childish Gambino
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...","This is America (skrrt, skrrt, woo), Don't cat...",Childish Gambino
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],"Ooh-ooh-ooh-ooh-ooh, tell somebody, You go tel...",Childish Gambino
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...","This is America (woo, ayy), Don't catch you sl...",Childish Gambino


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6304 entries, 0 to 6303
Data columns (total 5 columns):
song_owner      6304 non-null object
song_title      6304 non-null object
song_element    6304 non-null object
verse           6304 non-null object
artist          6304 non-null object
dtypes: object(5)
memory usage: 246.3+ KB


In [5]:
model_df = df.copy()

model_df = model_df[model_df['artist'] == 'Chance the Rapper']

In [6]:
model_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 468 entries, 316 to 5308
Data columns (total 5 columns):
song_owner      468 non-null object
song_title      468 non-null object
song_element    468 non-null object
verse           468 non-null object
artist          468 non-null object
dtypes: object(5)
memory usage: 21.9+ KB


In [7]:
model_df['verse_words'] = model_df['verse'].copy()
model_df['verse_words'] = model_df['verse_words'].str.lower()
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.replace(',',''))
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.split(' '))

In [8]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words
316,Childish Gambino,They Don't Like Me,[Verse 1: Chance The Rapper],"Eddy Scissorhands is workin' on my linin', Get...",Chance the Rapper,"[eddy, scissorhands, is, workin', on, my, lini..."
793,Childish Gambino,The Worst Guys (Live at Governor’s Ball),[Verse 2: Chance The Rapper],"You are the prototype, I love you like you lov...",Chance the Rapper,"[you, are, the, prototype, i, love, you, like,..."
824,Chance The Rapper,Favorite Song,[Verse 1: Chance The Rapper],"Chance, acid rapper, soccer, hacky sacker, Coc...",Chance the Rapper,"[chance, acid, rapper, soccer, hacky, sacker, ..."
825,Chance The Rapper,Favorite Song,[Chorus: Chance The Rapper],"This shit my favorite song, you just don't kno...",Chance the Rapper,"[this, shit, my, favorite, song, you, just, do..."
826,Chance The Rapper,Favorite Song,[Verse 2: Chance The Rapper],Young Rascal Flatts - young ass kid ass could ...,Chance the Rapper,"[young, rascal, flatts, -, young, ass, kid, as..."


In [9]:
from nltk.tokenize import word_tokenize
model_df['tokenize_verse'] = model_df['verse'].copy()
model_df['tokenize_verse'] = model_df['tokenize_verse'].str.lower()
model_df['tokenize_verse'] = model_df['tokenize_verse'].apply(lambda x: word_tokenize(x))

In [10]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
316,Childish Gambino,They Don't Like Me,[Verse 1: Chance The Rapper],"Eddy Scissorhands is workin' on my linin', Get...",Chance the Rapper,"[eddy, scissorhands, is, workin', on, my, lini...","[eddy, scissorhands, is, workin, ', on, my, li..."
793,Childish Gambino,The Worst Guys (Live at Governor’s Ball),[Verse 2: Chance The Rapper],"You are the prototype, I love you like you lov...",Chance the Rapper,"[you, are, the, prototype, i, love, you, like,...","[you, are, the, prototype, ,, i, love, you, li..."
824,Chance The Rapper,Favorite Song,[Verse 1: Chance The Rapper],"Chance, acid rapper, soccer, hacky sacker, Coc...",Chance the Rapper,"[chance, acid, rapper, soccer, hacky, sacker, ...","[chance, ,, acid, rapper, ,, soccer, ,, hacky,..."
825,Chance The Rapper,Favorite Song,[Chorus: Chance The Rapper],"This shit my favorite song, you just don't kno...",Chance the Rapper,"[this, shit, my, favorite, song, you, just, do...","[this, shit, my, favorite, song, ,, you, just,..."
826,Chance The Rapper,Favorite Song,[Verse 2: Chance The Rapper],Young Rascal Flatts - young ass kid ass could ...,Chance the Rapper,"[young, rascal, flatts, -, young, ass, kid, as...","[young, rascal, flatts, -, young, ass, kid, as..."


In [11]:
model_df['verse'] = model_df['verse'].str.lower()
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace("'",''))
model_df['verse'] = model_df['verse'].apply(lambda x: "".join([c if c.isalnum() else " " for c in x ]))
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace('  ',' '))

In [12]:
stopwords = ["yeah", "uh", "huh", "aw", "oh", "la", "ayy",
            "na", "ha", "ay", "skrrt",
            "em", "bla", "woo", "wu", "ya", "wala", "woop", "wrarrr", "ooh",
            "ba", "duh", "du", "1", "2", "3", "4" ,"5" ,"6", "7", "8", "9", "0"]

for index, row in model_df.iterrows():
    querywords = row['verse'].split()

    resultwords  = [word for word in querywords if word.lower() not in stopwords]
    row['verse'] = ' '.join(resultwords)

model_df.verse[:5]

316    eddy scissorhands is workin on my linin gettin...
793    you are the prototype i love you like you love...
824    chance acid rapper soccer hacky sacker cocky k...
825    this shit my favorite song you just dont know ...
826    young rascal flatts young ass kid ass could ra...
Name: verse, dtype: object

In [13]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
316,Childish Gambino,They Don't Like Me,[Verse 1: Chance The Rapper],eddy scissorhands is workin on my linin gettin...,Chance the Rapper,"[eddy, scissorhands, is, workin', on, my, lini...","[eddy, scissorhands, is, workin, ', on, my, li..."
793,Childish Gambino,The Worst Guys (Live at Governor’s Ball),[Verse 2: Chance The Rapper],you are the prototype i love you like you love...,Chance the Rapper,"[you, are, the, prototype, i, love, you, like,...","[you, are, the, prototype, ,, i, love, you, li..."
824,Chance The Rapper,Favorite Song,[Verse 1: Chance The Rapper],chance acid rapper soccer hacky sacker cocky k...,Chance the Rapper,"[chance, acid, rapper, soccer, hacky, sacker, ...","[chance, ,, acid, rapper, ,, soccer, ,, hacky,..."
825,Chance The Rapper,Favorite Song,[Chorus: Chance The Rapper],this shit my favorite song you just dont know ...,Chance the Rapper,"[this, shit, my, favorite, song, you, just, do...","[this, shit, my, favorite, song, ,, you, just,..."
826,Chance The Rapper,Favorite Song,[Verse 2: Chance The Rapper],young rascal flatts young ass kid ass could ra...,Chance the Rapper,"[young, rascal, flatts, -, young, ass, kid, as...","[young, rascal, flatts, -, young, ass, kid, as..."


In [14]:
def get_words(df):
    all_words = []
    
    for index, row in df['verse'].iteritems():
        new_words = re.findall(r"\b[a-z']+\b", unidecode(row))
        all_words += new_words
        
    return all_words

In [15]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
316,Childish Gambino,They Don't Like Me,[Verse 1: Chance The Rapper],eddy scissorhands is workin on my linin gettin...,Chance the Rapper,"[eddy, scissorhands, is, workin', on, my, lini...","[eddy, scissorhands, is, workin, ', on, my, li..."
793,Childish Gambino,The Worst Guys (Live at Governor’s Ball),[Verse 2: Chance The Rapper],you are the prototype i love you like you love...,Chance the Rapper,"[you, are, the, prototype, i, love, you, like,...","[you, are, the, prototype, ,, i, love, you, li..."
824,Chance The Rapper,Favorite Song,[Verse 1: Chance The Rapper],chance acid rapper soccer hacky sacker cocky k...,Chance the Rapper,"[chance, acid, rapper, soccer, hacky, sacker, ...","[chance, ,, acid, rapper, ,, soccer, ,, hacky,..."
825,Chance The Rapper,Favorite Song,[Chorus: Chance The Rapper],this shit my favorite song you just dont know ...,Chance the Rapper,"[this, shit, my, favorite, song, you, just, do...","[this, shit, my, favorite, song, ,, you, just,..."
826,Chance The Rapper,Favorite Song,[Verse 2: Chance The Rapper],young rascal flatts young ass kid ass could ra...,Chance the Rapper,"[young, rascal, flatts, -, young, ass, kid, as...","[young, rascal, flatts, -, young, ass, kid, as..."


In [16]:
get_words(model_df)

['eddy',
 'scissorhands',
 'is',
 'workin',
 'on',
 'my',
 'linin',
 'gettin',
 'spiffy',
 'for',
 'these',
 'bitches',
 'i',
 'be',
 'eyeing',
 'fredward',
 'mercury',
 'was',
 'playin',
 'while',
 'im',
 'ridin',
 'down',
 'the',
 'ryan',
 'in',
 'a',
 'scion',
 'and',
 'im',
 'flyin',
 'but',
 'its',
 'cool',
 'i',
 'got',
 'insurance',
 'on',
 'the',
 'run',
 'because',
 'i',
 'got',
 'warrants',
 'but',
 'its',
 'cool',
 'i',
 'got',
 'endurance',
 'in',
 'school',
 'i',
 'got',
 'suspended',
 'but',
 'in',
 'turn',
 'i',
 'got',
 'some',
 'earnings',
 'couldve',
 'spent',
 'it',
 'all',
 'on',
 'thursday',
 'but',
 'i',
 'saved',
 'it',
 'for',
 'my',
 'parents',
 'but',
 'i',
 'spent',
 'a',
 'bit',
 'on',
 'friday',
 'in',
 'the',
 'night',
 'i',
 'hang',
 'with',
 'donald',
 'im',
 'tired',
 'of',
 'mcdonalds',
 'want',
 'a',
 'chick',
 'that',
 'cook',
 'chicano',
 'that',
 'cant',
 'speak',
 'a',
 'lick',
 'of',
 'english',
 'and',
 'pronounce',
 'my',
 'name',
 'like',
 'ch

In [17]:
verses = list(model_df.verse.values)

In [18]:
verses[:10]

['eddy scissorhands is workin on my linin gettin spiffy for these bitches i be eyeing fredward mercury was playin while im ridin down the ryan in a scion and im flyin but its cool i got insurance on the run because i got warrants but its cool i got endurance in school i got suspended but in turn i got some earnings couldve spent it all on thursday but i saved it for my parents but i spent a bit on friday in the night i hang with donald im tired of mcdonalds want a chick that cook chicano that cant speak a lick of english and pronounce my name like chano she say something something spanish i look back like bitch i know tonight imma make decisions for life take some lucys boomers dark liq and light some might drive thats that shit right there that i dont like there that shit just had to say shouts to the niggas that passed away and shots for the niggas thats here right now and shots at the niggas that got away so sad to say but its all good niggas gon celebrate and wait for the day and w

In [19]:
def make_word_dictionary(verses):
    following = {}
    count = {}
    for verse in verses:
        words = verse.split()
        for index, word in enumerate(words[:-1]):
            # ensure the word is in the dictionary
            if word not in following:
                following[word] = []
                count[word] = 0

            # append the next word
            following[word].append(words[index + 1])
            count[word] += 1

        if words:
            last_word = words[-1]
            if last_word not in following:
                following[last_word] = []
                count[last_word] = 0
                
    return count, following

In [20]:
count, following = make_word_dictionary(verses)

In [21]:
def create_bigrams(verses):
    bigrams = {}
    for words in verses:
        words = words.split()
        words = [x for x in words if x]
        for w1, w2, w3 in zip(words[:-2], words[1:-1], words[2:]):
            bigram = (w1,w2)
            if bigram not in bigrams:
                bigrams[bigram] = [w3]
            else:
                bigrams[bigram].append(w3)   
    return bigrams

In [22]:
verses_bigrams = create_bigrams(verses)

In [23]:
def generate_text_from_bigram(following, bigram, num_words, seeds):    
    all_bigrams = list(bigram.keys())
    current_bigram = random.choice(seeds)
    text = [current_bigram]
    for _ in range(num_words - 1):
        if current_bigram in all_bigrams:
            
            next_word = random.choice(bigram[current_bigram])
        else:
            if following[text[-1]]:
                next_word = random.choice(following[text[-1]])
            else:
                continue
        text.append(next_word)
        current_bigram = (current_bigram[1], next_word)
    return ' '.join(text)

In [24]:
seeds = ["bitch", "love", "drugs", "cash", "guns", "club", "friends", "paper", "liquor", "apple",
         "blue", "red", "streets", "people", "brother", "cap"]

generated_raps = []
for _ in range(0,100):
    wordlist = generate_text_from_bigram(following, verses_bigrams, 33, seeds)
    generated_raps.append(wordlist)
    print(wordlist)
    print('\n')

guns to war with my blunt on my lips with the shits ive been waiting all night sledding coked out like dont change when you grow up i need space now i burned


liquor my fits and mismatch that get a pic you should flake on that lsd asked joseph about my deal he looked back said hell lets eat young rascal flatts young ass kid


brother all these things all all i know the difference in blessings and worldly possessions like my hugs with a pot of gold tourist grab a couple bucks take some niggas on a


love it down juke juke juke juke juke juke juke juke juke juke juke juke juke juke juke break it down break it down chi town break it down chi town break it


club is to find a gun range i had the shots that we should let know we the best i really meant well but i no hable ingles that language be that chef


drugs isnt close aint no activist im the illest so bitch lets fuck so i can sang with you ye aint know open mics open doors did i aint even that much closer


paper put this in your gir

club saying to her something told me these goofies mad though like why the devil damned you got me fucked up fucking with my windows down and i shot him batman gotham autumn


love you got me feeling upper echelon and i kiss her and watch black ink and being mary but never love hip hop that shit in tennessee cause i got shit to do


love im burning up my mic like its a simple message and it just dont get better it just not two thinkin back on my paper good watch captain save a hood hood


friends when i wait but it be ridin all my ladies bob that back this my jam im bout that jam im bout that jam im bout that jam if they ever get


apple pies lullabies and alibis the book dont end with the shits ive been warned bout i hope i hate money i hate crowded beaches i hate the kids i hate to love


friends when the praises go up good god the blessings come down when the going get rough im gonna be im gonna love life til im gone when the going get rough im


blue and gay cousins yall just gon s

In [25]:
generated_raps

['guns to war with my blunt on my lips with the shits ive been waiting all night sledding coked out like dont change when you grow up i need space now i burned',
 'liquor my fits and mismatch that get a pic you should flake on that lsd asked joseph about my deal he looked back said hell lets eat young rascal flatts young ass kid',
 'brother all these things all all i know the difference in blessings and worldly possessions like my hugs with a pot of gold tourist grab a couple bucks take some niggas on a',
 'love it down juke juke juke juke juke juke juke juke juke juke juke juke juke juke juke break it down break it down chi town break it down chi town break it',
 'club is to find a gun range i had the shots that we should let know we the best i really meant well but i no hable ingles that language be that chef',
 'drugs isnt close aint no activist im the illest so bitch lets fuck so i can sang with you ye aint know open mics open doors did i aint even that much closer',
 'paper put th

In [26]:
columns = ['markov_generated_verses']

chance_markov_df = pd.DataFrame(generated_raps, columns=columns)

In [27]:
chance_markov_df['artist'] = 'Chance the Rapper'

chance_markov_df

,markov_generated_verses,artist
0,guns to war with my blunt on my lips with the ...,Chance the Rapper
1,liquor my fits and mismatch that get a pic you...,Chance the Rapper
2,brother all these things all all i know the di...,Chance the Rapper
3,love it down juke juke juke juke juke juke juk...,Chance the Rapper
4,club is to find a gun range i had the shots th...,Chance the Rapper
5,drugs isnt close aint no activist im the illes...,Chance the Rapper
6,paper put this in your girl way wetter than yo...,Chance the Rapper
7,club saying to her something told me these goo...,Chance the Rapper
8,bitch wanna i wanna thank l boog mama i wanna ...,Chance the Rapper
9,club saying to her something told me that i wa...,Chance the Rapper


In [28]:
with open('data/chance_markov_raps.pkl', 'wb') as picklefile:
    pickle.dump(chance_markov_df, picklefile)